# Vector Alternative

## Earlier work

Previously I used Hall's method for applying DP to functions.

I found a bound which applied to any test point $x_*$. I then tried to alter this to just consider the greatest bound for the test points used (i.e. calculate the exact bound for each test point, and use that to calculate $\Delta$). But it turns out I can't use as the sensitivity the test point with the highest sensitivity as I need to bound the whole function: It is almost certain that combining the results of more than one test point in a joint probability distribution will leak more information, I think is the issue.

I also looked into manipulating the covariance function by: 

 - Altering the noise variance of each training point
 - Scaling the kernel by the cloaking function
 
> Note: I've not investigated the effect of using such modified covariance functions much: it seems to behave quite strangely. The problem is its interaction with the sensitivity. Maybe something to look into later.

## Noise direction

It is worth reasoning about the direction of the noise that's added, and compare it to the effect of modifying a training point. I think the sensitivity in the earlier methods needs to be quite high because the noise added (sampled from the *prior*) is not necessarily in the direction a perturbation in a training output would cause.

Consider the simple case of two training and two test points as in this figure;

#### Figure 1
<img src="demo_problem_with_standard_fn_noise.png" />

Put simply: We're interested in the effect of changing the training outputs on the test outputs. Specifically what's the largest change in the mean of the prediction at the test points, given a unit change in one of the training points?

Using the method developed in the earlier version of the paper we add noise with correlations defined by the default kernel (i.e. from the **prior**). In the following figure the shaded contour plot shows the distribution of DP noise added to two nearby test points. The two axes are for the two test points above (x-axis, left point; y-axis, right point). 

The lined contours are the same noise, but for when the training data has been *perturbed*, as in the above figure. The y-axis represents the right test point, and, as this has moved due to the perturbation, the gaussian has shifted upwards. Because the other training point wasn't affected the gaussian has not shiften along the x-axis.

#### Figure 2
<img src="problem.png" />

If the other training point were to move, the gaussian would be shifted rightward and *downward* (think about how the right test point would move if the left training point moved up).

In this case I've reduced the $\Delta$ of the DP so the two gaussians don't overlap too much. But for DP to exist the $\Delta$ must be large enough that the two distributions overlap sufficiently. It seems to me that the direction of the Gaussian noise is wrong, and that it needs to point in the direction which means the $\Delta$ is as small as possible.

To this end, I went back to an earlier step in the Hall paper, for a bound on a vector rather than a function. This means, I think, I can add whatever gaussian I want (the covariance of the gaussian doesn't have to match the kernel).

Copied from the paper:

> Proposition 3: Suppose that, for a positive definite symmetric matrix $M \in \mathbb{R}^{d \times d}$, the family of vectors $\{\mathbf{v}_D : D \in \mathcal{D}\} \subset \mathbb{R}^d$
>
> $$sup_{D \sim {D'}} ||M^{-1/2} (\mathbf{v}_D - \mathbf{v}_{D'})||_2 \leq \Delta\;\;\;\;\;(1)$$
>
> Then the randomized algorithm which, for input database D outputs
>
> $$\tilde{\mathbf{v}_D} = \mathbf{v}_D + \frac{c(\delta)\Delta}{\alpha}Z\;\;\;\;\;(2)$$
>
> where $$Z \sim \mathcal{N}_d(0,M)$$
>
> achieves $(\varepsilon, \delta)$-DP whenever,
>
> $$c(\delta) \geq \sqrt{2 log \frac{2}{\delta}}$$

We want to construct $M$ so that it has greatest covariance in those directions most affected by changes in test points. This has the intuitive result that it will mask those changes, and the mathematical result that its square-root inverse, $M^{-\frac{1}{2}}$, will scale-down the test-perturbation $(\mathbf{v}_D - \mathbf{v}_{D'})$ most in those 'directions' in which it's greatest.

We build the matrix $K$ to represent the covariance between all training points, with the sample variance built in, i.e. $K_{i,i} = k(x_i,x_i) + \sigma^2$.

The prediction at test point $x_*$ has mean calculated as: $\mathbf{k}_* K^{-1} \mathbf{y}$ where $\mathbf{k}_*$ is the covariance between test point $x_*$ and all the training points. We can find the predictions for all test points $\mathbf{x}_*$ if we consider the matrix $K_{*N}$, the covariance between all test and training points.

We build another matrix $C$, the cloaking-matrix, or sensitivity matrix. This describes how much each test point will be altered for a unit change in each training point.

$$C = K_{*N} K^{-1}$$

> Detour: If we assume that (training) element $i$ of $\mathbf{y}$ was perturbed by $\Delta_y$, so a new vector $\mathbf{y}'$ is equal to $\mathbf{y}$, except for one element, then the prediction for all the test points was be altered by $K_{*N} K^{-1} (\mathbf{y}-\mathbf{y}') = K_{*N} [K^{-1}]_{:i} \Delta_y$. Because the calculations which follow are linear wrt $\Delta_y$, we can consider the sensitivity for a unit perturbation for now, and later scale by $\Delta_y$.

(reminder: $K_{*N}$, the covariance between all test and training points. $K$ is the covariance between training points)

One column, i, of $C$, represents the effect of training point $x_i$'s perturbation by 1 on all test points. Of interest here then is the variance and covariance of all the test points when manipulating training point $i$.

We can find a covariance matrix which describes these simply by considering the total set of test point mean predictions.

We want the covariance across test points, given a change in a training point.

We can rewrite earlier expressions as: (Todo: show)

$$\mathbf{y}_*' = \mathbf{y}_* + C_{:i} (y_i' - y_i)\;\;\;\;\;\;(3)$$

where $C = K_{*N} K^{-1}$, and $\mathbf{y}_* = K_{*N} K^{-1} \mathbf{y}$ (the normal prediction).

$$\langle (y_*' - \langle y_*' \rangle)( y_*' - \langle y_*' \rangle)^\top \rangle \;\;\;\;\;\; (4) $$

We define $\langle y_i' \rangle = y_i$, i.e. the perturbation we expect is symmetric. 

So $\langle y_i' - y_i \rangle = 0$

so $\langle c_{:i} (y_i'-y_i) \rangle = 0$

so using (3):

$\langle y_*' \rangle = \langle y_* \rangle + 0$

so substituting into (4):

$\langle (y_*' - y_*)( y_*' - y_*)^\top \rangle$

which using (3) is:

$\langle (c_{:i}(y_i' - y_i))(c_{:i}(y_i' - y_i))^\top \rangle$

$c_{:i} \langle ? \rangle c_{:i}^{\top}$


$\bar{\mathbf{y}} = \mathbf{K}_* K^{-1} \mathbf{y}$ where $\mathbf{k}_*$



